In [13]:
import pickle
import glob
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss
import lightgbm as lgb

# import time
# print('sleeping')
# time.sleep(14400)
# print('sleep done =======================')

# load feats
train_x,test_x = [],[]
for feat in sorted(glob.glob('../features/*.pkl')):
    if '3_feat' in feat or 'tfidf' in feat:
        continue
    print('file path',feat)
    a,b = pickle.load(open(feat,'rb'))
    print(a.shape,b.shape)
    train_x.append(a)
    test_x.append(b)
train_x = np.hstack(train_x)
test_x = np.hstack(test_x)
print(train_x.shape)
    
# load y
train = pd.read_csv("../input/train.csv")
list_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
train_y = train[list_classes].values.astype('int')
print(train_x.shape)

from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import roc_auc_score
def simple_ens(model_name,k=3,rnd=233,lr=0.05,feature_fraction=0.9,bagging_fraction=0.9,
               bag_frec=3,met='binary_logloss',max_d=3):
    kf = StratifiedKFold(n_splits=k, shuffle=True, random_state=rnd)
    test_pred = np.zeros((153164,6))
    cls_auc_res = [0,0,0,0,0,0]
    all_train_loss_l,all_val_loss_l = 0,0
    all_train_auc_l,all_val_auc_l = 0,0
    
    for i in range(6):
        val_loss_l,train_loss_l = 0,0
        val_auc_l,train_auc_l = 0,0
        fold_cnt = 0
        for train_index, test_index in kf.split(train_x,train_y[:,i]):
            # x,y
            curr_x,curr_y = train_x[train_index],train_y[train_index]
            hold_out_x,hold_out_y = train_x[test_index],train_y[test_index]
            d_test = test_x

            # share params
            params = {
                    'application': 'binary',
                    #'num_leaves': 8,
                    #'lambda_l1': 1,
                    'lambda_l2': 1.0,
                    'max_depth': max_d,
                    'metric': met, # or auc
                    'data_random_seed': 2,
                    'learning_rate':lr,
                    # 'bagging_fraction': bagging_fraction,
                    # 'bagging_freq':bag_frec,
                    'feature_fraction': feature_fraction,

                    }
            if met == 'auc':
                s_round = 60
            else:
                s_round = 30
            # train for each class
            d_train = lgb.Dataset(curr_x, curr_y[:,i])
            d_valid = lgb.Dataset(hold_out_x, hold_out_y[:,i])
            watchlist = [d_train, d_valid]
            model = lgb.train(params,
                      train_set=d_train,
                      num_boost_round=2000,
                      valid_sets=watchlist,
                      early_stopping_rounds=s_round,
                      verbose_eval=None)
            print(fold_cnt,'fold: ',end='')
            fold_cnt += 1
            try:
                train_pred = model.predict(curr_x)
                tmp_test_pred = model.predict(hold_out_x)
                
                curr_train_loss = log_loss(curr_y[:,i],train_pred)
                curr_val_loss = log_loss(hold_out_y[:,i],tmp_test_pred)
                
                curr_train_auc = roc_auc_score(curr_y[:,i],train_pred)
                curr_val_auc = roc_auc_score(hold_out_y[:,i],tmp_test_pred)
                
                print('ls',curr_train_loss,curr_val_loss,'auc',curr_train_auc,curr_val_auc)
                val_loss_l += curr_val_loss
                train_loss_l += curr_train_loss
                val_auc_l += curr_val_auc
                train_auc_l += curr_train_auc
            except:
                pass
            curr_test_pred = model.predict(d_test)
            test_pred[:,i] += curr_test_pred
            
        # avg k fold
        train_loss_l = train_loss_l/k
        val_loss_l = val_loss_l/k
        train_auc_l = train_auc_l/k
        val_auc_l = val_auc_l/k
        print(list_classes[i], lr, feature_fraction, max_d)
        print('this class avg train',train_loss_l,'avg val',val_loss_l)
        print('this class auc train',train_auc_l,'auc val',val_auc_l)
        cls_auc_res[i] = val_auc_l
        
        
        # avg 6 class
        all_train_loss_l += train_loss_l/6
        all_val_loss_l += val_loss_l/6
        all_train_auc_l += train_auc_l/6
        all_val_auc_l += val_auc_l/6
        print('========================')
    test_pred = test_pred/k
    print('all loss avg',all_train_loss_l,all_val_loss_l)
    print('all auc avg',all_train_auc_l,all_val_auc_l)
    print('=======================================================')
    return test_pred, cls_auc_res

print('done')

file path ../features/fasttext_cnn2d_5_feat.pkl
(159571, 6) (153164, 6)
file path ../features/fasttext_cnn_gru_5_feat.pkl
(159571, 6) (153164, 6)
file path ../features/fasttext_cnn_v1_5_feat.pkl
(159571, 6) (153164, 6)
file path ../features/fasttext_cnn_v2_5_feat.pkl
(159571, 6) (153164, 6)
file path ../features/fasttext_cudnn_gru_5_feat.pkl
(159571, 6) (153164, 6)
file path ../features/fasttext_gru_v1_5_feat.pkl
(159571, 6) (153164, 6)
file path ../features/fasttext_lstm_v1_5_feat.pkl
(159571, 6) (153164, 6)
file path ../features/glove_cnn2d_5_feat.pkl
(159571, 6) (153164, 6)
file path ../features/glove_cnn_gru_5_feat.pkl
(159571, 6) (153164, 6)
file path ../features/glove_cnn_v1_5_feat.pkl
(159571, 6) (153164, 6)
file path ../features/glove_cnn_v2_5_feat.pkl
(159571, 6) (153164, 6)
file path ../features/glove_cudnn_gru_5_feat.pkl
(159571, 6) (153164, 6)
file path ../features/glove_gru_v1_5_feat.pkl
(159571, 6) (153164, 6)
file path ../features/glove_lstm_v1_5_feat.pkl
(159571, 6) (15

In [16]:
lgb_res,tmp_auc_res = simple_ens('lgb',10,666,0.05,0.45)
sample_submission = pd.read_csv("../input/sample_submission.csv")
sample_submission[list_classes] = lgb_res
sample_submission.to_csv("../results/lgb_log_csv_fold10_stratified.gz", index=False, compression='gzip')
print(sample_submission.head())
print('save done')

# add 7 base fasttext NN models
# all loss avg 0.0319116484218 0.0358161833583 all auc avg 0.994546891137 0.991249510282 PUB 9866

# rm tfidf test
# all loss avg 0.0319555637279 0.0357756335619 all auc avg 0.994512718368 0.991251471241

# change to stratified
# all loss avg 0.0316412744167 0.0357232681944 all auc avg 0.994584962017 0.991293307537 pub 9866

# change to base model 5 fold, add word batch, tilli, lgb feat
# feat dim 217
# all loss avg 0.031983129767200906 0.035387924581 all auc avg 0.9945427088311004 0.99188514127 PUB 9870

# add more base models
# feat dim 247, all loss avg 0.031885221777487156 0.0353958  all auc avg 0.9945986685511962 0.9919487331094685

# change early stopping to 30, and test later part
# all loss avg 0.03208696729295824 0.035406264613808025 all auc avg 0.9945261773637045 0.991946174432887

# add fasttext lstm v1
# all loss avg 0.031977558955160815 0.03537909655655411 all auc avg 0.9945299073418443 0.9919307178575658

# add muse base model, feat dim 295, lower loss, but lower auc
# all loss avg 0.03196610276261484 0.03530962013098028 all auc avg 0.9945635742334386 0.9918682952070021

# fix pool gru fold to 5, and adj params
# all loss avg 0.03172365669814678 0.03528054768190897 all auc avg 0.9946437910139179 0.991903618166854

# updated pool gru v2
# all loss avg 0.03204678384090053 0.035272185628381025 all auc avg 0.9944980867573662 0.9919033708443966

# updated other feat, change some cnt to ratio, a bit worse
# all loss avg 0.03197308230179816 0.03527747086566205 all auc avg 0.9945654974291834 0.991902658184264

# updated pool gru v2 10 fold PUB 9870
# all loss avg 0.03188938973578164 0.035151701851945265 all auc avg 0.9945816630869728 0.9919824289801534

# rm lr, mnb feat1
# worse all loss avg 0.032097370918022436 0.03520186226431002
# all auc avg 0.9944811076726177 0.9919214297276806

# add ridge , change lr,mnb,ridge to fold 6
# all loss avg 0.03187703661408994 0.0351472518210873 all auc avg 0.9946115814252721 0.9920990121249438

# ridge, lr, mnb fold 10, feat frac 0.6
# all loss avg 0.031754244562510095 0.03510006533423048 all auc avg 0.9946626820706096 0.9921195758845225

# lgb v1 feat fold 10, 5 fold is better, change feat file back
# all loss avg 0.031846113030824186 0.03510235514884163 all auc avg 0.9946147807624219 0.9920710159886731

# feat frac 0.45 PUB 9871
# all loss avg 0.03178085998781136 0.035091249281527216 all auc avg 0.9946391992503673 0.9921624859169091

0 fold: ls 0.0701461797096452 0.07801615635891698 auc 0.9906122184812396 0.9869962364393128
1 fold: ls 0.07037840323424928 0.071295365077439 auc 0.990521975787274 0.989788057353983
2 fold: ls 0.07094377319551964 0.07530325152593936 auc 0.9902848024854317 0.988670586060873
3 fold: ls 0.06614382554676579 0.07264003164035851 auc 0.991893373728453 0.9894589949462828
4 fold: ls 0.06920242084187422 0.07761195746407645 auc 0.9908886114964512 0.98796414137687
5 fold: ls 0.0687686656784246 0.07544745851319139 auc 0.9910595491009526 0.9879742046522069
6 fold: ls 0.0684518988156945 0.07722394770028321 auc 0.9911430051710824 0.9880209852835022
7 fold: ls 0.07104156493308908 0.06986341907854433 auc 0.990267760685616 0.9905210975551211
8 fold: ls 0.06759409867709222 0.07391844104036682 auc 0.9914021848395582 0.9894022739048028
9 fold: ls 0.06997786109459782 0.07696660222007166 auc 0.9905878368975015 0.9880193389665287
toxic 0.05 0.45 3
this class avg train 0.06926486917269523 avg val 0.0748286630619

In [14]:
def special_ens(model_name,k=3,rnd=233):
    kf = StratifiedKFold(n_splits=k, shuffle=True, random_state=rnd)
    test_pred = np.zeros((153164,6))
    all_train_loss_l,all_val_loss_l = 0,0
    all_train_auc_l,all_val_auc_l = 0,0
    
    params_list = [
        [0.05, 3,0.4], # depth should be 3
        [0.075,3,0.6],
        [0.095,3,0.6],
        [0.05, 3,0.6],
        [0.075,3,0.4],
        [0.095,3,0.4],
    ]
    
    for i in range(6):
        val_loss_l,train_loss_l = 0,0
        val_auc_l,train_auc_l = 0,0
        fold_cnt = 0
        
        # special params
        params = {
                'application': 'binary',
                #'num_leaves': 8,
                #'lambda_l1': 1,
                'lambda_l2': 1.0,
                'max_depth': params_list[i][1],
                'metric': 'binary_logloss', # or auc
                'data_random_seed': 2,
                'learning_rate':params_list[i][0],
                'feature_fraction': params_list[i][2],

                }
        print(params)
            
        for train_index, test_index in kf.split(train_x,train_y[:,i]):
            # x,y
            curr_x,curr_y = train_x[train_index],train_y[train_index]
            hold_out_x,hold_out_y = train_x[test_index],train_y[test_index]
            d_test = test_x
            s_round = 30
            # train for each class
            d_train = lgb.Dataset(curr_x, curr_y[:,i])
            d_valid = lgb.Dataset(hold_out_x, hold_out_y[:,i])
            watchlist = [d_train, d_valid]
            model = lgb.train(params,
                      train_set=d_train,
                      num_boost_round=2000,
                      valid_sets=watchlist,
                      early_stopping_rounds=s_round,
                      verbose_eval=None)
            print(fold_cnt,'fold: ',end='')
            fold_cnt += 1
            try:
                train_pred = model.predict(curr_x)
                tmp_test_pred = model.predict(hold_out_x)
                
                curr_train_loss = log_loss(curr_y[:,i],train_pred)
                curr_val_loss = log_loss(hold_out_y[:,i],tmp_test_pred)
                
                curr_train_auc = roc_auc_score(curr_y[:,i],train_pred)
                curr_val_auc = roc_auc_score(hold_out_y[:,i],tmp_test_pred)
                
                print('ls',curr_train_loss,curr_val_loss,'auc',curr_train_auc,curr_val_auc)
                val_loss_l += curr_val_loss
                train_loss_l += curr_train_loss
                val_auc_l += curr_val_auc
                train_auc_l += curr_train_auc
            except:
                pass
            curr_test_pred = model.predict(d_test)
            test_pred[:,i] += curr_test_pred
            
        # avg k fold
        train_loss_l = train_loss_l/k
        val_loss_l = val_loss_l/k
        train_auc_l = train_auc_l/k
        val_auc_l = val_auc_l/k
        print(list_classes[i])
        print('this class avg train',train_loss_l,'avg val',val_loss_l)
        print('this class auc train',train_auc_l,'auc val',val_auc_l)
        
        
        # avg 6 class
        all_train_loss_l += train_loss_l/6
        all_val_loss_l += val_loss_l/6
        all_train_auc_l += train_auc_l/6
        all_val_auc_l += val_auc_l/6
        print('========================')
    test_pred = test_pred/k
    print('all loss avg',all_train_loss_l,all_val_loss_l)
    print('all auc avg',all_train_auc_l,all_val_auc_l)
    print('=======================================================')
    return test_pred

print('done')

lgb_res = special_ens('lgb',10,666)
sample_submission = pd.read_csv("../input/sample_submission.csv")
sample_submission[list_classes] = lgb_res
sample_submission.to_csv("../results/lgb_log_csv_fold10_stratified_special.gz", index=False, compression='gzip')
print(sample_submission.head())
print('save done')

# best params changed when base models changed
# all loss avg 0.03111512578966158 0.03534320053008906 all auc avg 0.9948524214184179 0.9918450388634261

# change lr, ridge, mnb fold to 10, train loss too low ?
# all loss avg 0.030612393454975732 0.035095753229703264 all auc avg 0.9950485697696142 0.9921210772750391

done
{'feature_fraction': 0.4, 'max_depth': 4, 'learning_rate': 0.05, 'lambda_l2': 1.0, 'application': 'binary', 'metric': 'binary_logloss', 'data_random_seed': 2}
0 fold: ls 0.06696321505466694 0.07808999299738749 auc 0.9916958534307556 0.9869048654486285
1 fold: ls 0.06303855556133289 0.07096451163531511 auc 0.992889441615493 0.9899266767052445
2 fold: ls 0.06722168931452523 0.07542947440710139 auc 0.9915968136610711 0.9886690458458588
3 fold: ls 0.06143346454733205 0.0726748192207435 auc 0.9933412730767426 0.9893768199452407
4 fold: ls 0.06543866677533601 0.07752241946965202 auc 0.9921329577438122 0.9879959630853676
5 fold: ls 0.06370754273564935 0.0750665015567107 auc 0.9927020839766838 0.988260192058063
6 fold: ls 0.0631175325855247 0.07702720660208447 auc 0.9928248474882658 0.9880800956935891
7 fold: ls 0.06778323722881177 0.06974333061674629 auc 0.9914154024712916 0.990567609429725
8 fold: ls 0.06771805583936838 0.0738967994947421 auc 0.9914450150016623 0.9892846795551705
9 fold

In [3]:
# find best params for each column, early stopping = 30

best_pred = np.zeros((153164,6))
val_auc_res = [0,0,0,0,0,0]
for lr in [0.095,0.075,0.05]:
    for max_d in [3]:
        for s_rate in [0.4,0.48,0.56,0.62]:
            print('learning rate',lr,'max depth',max_d,'feature fraction',s_rate)
            lgb_res,tmp_auc_res = simple_ens('lgb',k=10,rnd=666,lr=lr,
                                 feature_fraction=s_rate,bagging_fraction=0.9,
                                 bag_frec=3,met='binary_logloss',max_d=max_d)
            # check for each cls
            for i in range(6):
                # find better params for this class
                if tmp_auc_res[i] > val_auc_res[i]:
                    val_auc_res[i] = tmp_auc_res[i]
                    best_pred[:,i] = lgb_res[:,i]
                    print('FIND BETTER PARAMS',lr,max_d,s_rate,list_classes[i])
            print('TEST PARAM DONE ------------------------------------------')

print(val_auc_res)
print(np.mean(val_auc_res))
sample_submission = pd.read_csv("../input/sample_submission.csv")
sample_submission[list_classes] = best_pred
sample_submission.to_csv("../results/lgb_grid_search_fold10_stratified.gz", index=False, compression='gzip')
print(sample_submission.head())
print('save done')
                
            
# best auc params
# toxic 0.05,4,0.5
# severe toxic 0.075 3 0.5
# obs 0.075 3 0.6
# threat 0.095 3 0.5
# insult 0.075 0.5 4
# hate 0.05 0.5 3

# TEST PARAM DONE ------------------------------------------
# [0.9884873039634368, 0.9920148292218363, 0.9954573112511824, 
#  0.994769628570376, 0.9898761527824232, 0.9912880359235366]
# 0.9919822102854652 PUB 9870

# updated pool gru v2 10 fold
# TEST PARAM DONE ------------------------------------------
# [0.988653301673445, 0.9921696955504364, 0.995510773279133, 
#  0.9948578115535328, 0.9899677533084541, 0.9914911558115035]
# 0.9921084151960842

learning rate 0.095 max depth 3 feature fraction 0.4
0 fold: ls 0.07061737961759704 0.07775928680520683 auc 0.9904295713822548 0.9870631451915393
1 fold: ls 0.06978029561549126 0.07145282016458328 auc 0.9907204797236494 0.9897310693984644
2 fold: ls 0.06961053786924241 0.07523377254243167 auc 0.9907552553794691 0.9886870301211695
3 fold: ls 0.07043228685873615 0.0734208127397867 auc 0.9905142818080588 0.9890545073033372
4 fold: ls 0.06932434768590719 0.07776492803204109 auc 0.9907917891506477 0.9878860829979059
5 fold: ls 0.07055630326132173 0.07581794629202872 auc 0.9904214584692226 0.9879176327259891
6 fold: ls 0.06981054734403931 0.07766761434216077 auc 0.9906565612464159 0.9878052594847275
7 fold: ls 0.07061174620035217 0.06981370568054665 auc 0.9903866606261856 0.9904415831028254
8 fold: ls 0.06576400798133654 0.0735997702164948 auc 0.9919821907448325 0.989493801658044
9 fold: ls 0.06968508568127503 0.07721907256609754 auc 0.9906951976357643 0.988006736333839
toxic 0.095 0.4 3
thi

toxic 0.095 0.5 3
this class avg train 0.06878802960203327 avg val 0.07498242051939999
this class auc train 0.9910007938217443 auc val 0.9886150639563438
0 fold: ls 0.018159802255339617 0.021012651452617513 auc 0.9940554787704652 0.9913687650335486
1 fold: ls 0.017954455127797458 0.020284301859314537 auc 0.9942023060556717 0.9919792062286366
2 fold: ls 0.018213842828436043 0.0209621094696875 auc 0.9940441983139063 0.9912619477149007
3 fold: ls 0.01584533353091705 0.019540398130610783 auc 0.9959977194999663 0.9928143594125839
4 fold: ls 0.018007367231727817 0.020736782924777014 auc 0.9941803725153282 0.9915463982782631
5 fold: ls 0.018080494744634818 0.02003000282874693 auc 0.994113466324198 0.9922173891926452
6 fold: ls 0.018073144929191612 0.01929794509591248 auc 0.9941192211293304 0.9927408396546913
7 fold: ls 0.017824278721464287 0.020262629120251886 auc 0.994327291359605 0.992093475275737
8 fold: ls 0.01755797206719156 0.021016414892181767 auc 0.9945646412489446 0.9911419372279507


0 fold: ls 0.03514853960801476 0.0370275268264882 auc 0.9962598907484033 0.9956900618888007
1 fold: ls 0.036013710087676995 0.037084881271528085 auc 0.9960683918908128 0.995500797346381
2 fold: ls 0.03489600059230024 0.03907963189169119 auc 0.9963038537717759 0.9953300171343906
3 fold: ls 0.03431183859790538 0.034164607880901796 auc 0.9964340804109733 0.9963843929821045
4 fold: ls 0.03505906207862771 0.04210810790879396 auc 0.9962781029653999 0.994504621900069
5 fold: ls 0.033719796603668015 0.037203789702455976 auc 0.996569700970018 0.9956073937871389
6 fold: ls 0.03541456005948037 0.03768331118364482 auc 0.9961931283017877 0.9953916476893632
7 fold: ls 0.0354221965850771 0.03827718830814151 auc 0.9961934159581747 0.9955228181843812
8 fold: ls 0.03416260004031948 0.03874712044178929 auc 0.9964765500966313 0.9952300926259472
9 fold: ls 0.03498723626067744 0.0384949102345425 auc 0.9962872624953467 0.9954128447559957
obscene 0.095 0.6 3
this class avg train 0.03491355405137475 avg val 0.

1 fold: ls 0.0038013991340242375 0.006108898674322724 auc 0.9994160099881937 0.9971899224806201
2 fold: ls 0.0037589476260691188 0.007289763192891955 auc 0.9994233432609985 0.995045045045045
3 fold: ls 0.004109614120580821 0.006358073273365257 auc 0.9992867284768729 0.9900927410061392
4 fold: ls 0.003965049590243477 0.006916370464727435 auc 0.999318725036187 0.9964629559787961
5 fold: ls 0.004086233014601164 0.0070535206985022 auc 0.9992260811913504 0.9965520041066902
6 fold: ls 0.003308370786333995 0.006387726600850115 auc 0.9996330140823841 0.9946741362331595
7 fold: ls 0.0039049277734833298 0.007742913868505792 auc 0.9993512738203827 0.9954925932910093
8 fold: ls 0.004337287688237121 0.007204515948587411 auc 0.9989705051362205 0.9945113364173632
9 fold: ls 0.004198239938376271 0.007446822627362225 auc 0.9991587820153337 0.9907719837426429
threat 0.095 0.4 4
this class avg train 0.003918497297788091 avg val 0.0069072382388765076
this class auc train 0.999322898586524 auc val 0.994803

2 fold: ls 0.047591547477326364 0.054944464992350624 auc 0.9926349646154158 0.9892255788040114
3 fold: ls 0.04842828017215234 0.05158870685802453 auc 0.9923479645622364 0.9904725296052522
4 fold: ls 0.04846112703415528 0.05527230396648496 auc 0.9923293834744886 0.9890156353476716
5 fold: ls 0.04721714684236866 0.05201066460414472 auc 0.9927966726654223 0.9907300756652712
6 fold: ls 0.045944912027801536 0.05625684630763563 auc 0.9932027809035558 0.9891576896910711
7 fold: ls 0.046565843161742844 0.05549654089621556 auc 0.9930177096265893 0.9895397077718945
8 fold: ls 0.04909823296561537 0.05220401374234562 auc 0.9921287875637712 0.9907386520174271
9 fold: ls 0.04911932742586718 0.05482416373755757 auc 0.992108977390389 0.9890637906524233
insult 0.095 0.5 4
this class avg train 0.04735126608516822 avg val 0.05362783071658792
this class auc train 0.9927312461421902 auc val 0.9899196566541493
0 fold: ls 0.013418849725249052 0.017224813665512078 auc 0.9966459934537784 0.9925674727389554
1 f

...

5 fold: ls 0.014355441234641544 0.01634289607596243 auc 0.9958385082457669 0.9936171750106124
6 fold: ls 0.012722564631529154 0.016538210464715004 auc 0.9970271015464583 0.9905985800997181
7 fold: ls 0.015354052466559083 0.01882433270090625 auc 0.9950093503875517 0.9867807916034396
8 fold: ls 0.013107808655497771 0.01652880738657252 auc 0.996788159230089 0.993053598525905
9 fold: ls 0.014790044190966934 0.016499679584218798 auc 0.9954886733278634 0.9924664896307536
identity_hate 0.05 0.4 3
this class avg train 0.014373468975760712 avg val 0.01740947366445552
this class auc train 0.995804141868469 auc val 0.9910499175426264
all loss avg 0.031856041454253296 0.03513541348068197
all auc avg 0.9946147646006687 0.9920073338661338
TEST PARAM DONE ------------------------------------------
learning rate 0.05 max depth 3 feature fraction 0.5
0 fold: ls 0.07048505638781383 0.0780099928470403 auc 0.9904824998716626 0.9870710274683758
1 fold: ls 0.06833916094072293 0.0711782474174137 auc 0.991200

4 fold: ls 0.06779634010274735 0.0775894900790881 auc 0.9913295559260866 0.9879393004990116
5 fold: ls 0.06771584933002278 0.0754902409286598 auc 0.9914162533481393 0.9879212138014467
6 fold: ls 0.0696782834291993 0.07732314130220892 auc 0.9907517283683197 0.9879465533100652
7 fold: ls 0.07243148939004662 0.07013710158949128 auc 0.9897586566746034 0.9904187351644234
8 fold: ls 0.06931333111402917 0.0739117954710366 auc 0.9908082300667973 0.9893665513344442
9 fold: ls 0.06915516424801603 0.07706986383730047 auc 0.990849452293511 0.9879574138001457
toxic 0.05 0.6 3
this class avg train 0.06908397034881779 avg val 0.07493585447704923
this class auc train 0.9909173787193553 auc val 0.9886182191066398
0 fold: ls 0.017461971862011476 0.021008241444619255 auc 0.994624405398876 0.9913280162045828
1 fold: ls 0.017533005558762325 0.020172782314556877 auc 0.994568103593753 0.992147740220281
2 fold: ls 0.01775793228721981 0.020899814505396998 auc 0.9943894518443118 0.9914103051019116
3 fold: ls 0.

6 fold: ls 0.016015497627088916 0.019125700051658517 auc 0.9959741693260806 0.9929506557848935
7 fold: ls 0.016044869451937768 0.019966564893646573 auc 0.9958974877699873 0.992364603899395
8 fold: ls 0.016595493340221607 0.02098357257697114 auc 0.9954991364900547 0.9912239526412745
9 fold: ls 0.016278025664388514 0.018811218663785788 auc 0.9957811038190492 0.9933782507067858
severe_toxic 0.05 0.4 4
this class avg train 0.016174724199790012 avg val 0.02019146207129583
this class auc train 0.995813929170691 auc val 0.9920895805665481
0 fold: ls 0.03314032902166893 0.037003570480217844 auc 0.9967201022904737 0.9957092467513959
1 fold: ls 0.03387676066319782 0.03687247409825699 auc 0.9965711759242469 0.9956036908543411
2 fold: ls 0.03188051334978406 0.038884671126702054 auc 0.9969776968351919 0.9953860093158461
3 fold: ls 0.033643161367225234 0.034106174620050826 auc 0.9966145623130644 0.9963972359440046
4 fold: ls 0.03219392790444366 0.042083611130430534 auc 0.9969198445429466 0.994540488

7 fold: ls 0.033344387061391395 0.03804373328126119 auc 0.9966774294955646 0.9955923581244264
8 fold: ls 0.03268275806628089 0.03867928101200202 auc 0.9968258195809084 0.9952607121265752
9 fold: ls 0.030584679229279953 0.038364938710532204 auc 0.9972590874293238 0.9954230372147053
obscene 0.05 0.5 4
this class avg train 0.03280223500077574 avg val 0.037883309361659644
this class auc train 0.9967935031218136 auc val 0.9954859512861383
0 fold: ls 0.0038554395256276673 0.006524287101203074 auc 0.9993692086026192 0.9972056358684265
1 fold: ls 0.004116937358939139 0.0060513252618656 auc 0.9992509179584622 0.9970877854598785
2 fold: ls 0.004092378178821716 0.0072964900206976616 auc 0.9992538740064145 0.9951615860046092
3 fold: ls 0.004012140265436217 0.006502575053762169 auc 0.999337955455592 0.9905956010224821
4 fold: ls 0.003998196872581158 0.006730499869369558 auc 0.9993288437450463 0.9967026002053345
5 fold: ls 0.0041270424861729075 0.006957061605568035 auc 0.9992173430446951 0.996592599

8 fold: ls 0.003873224812447109 0.0071750183396012196 auc 0.9993701337094788 0.9941020939572541
9 fold: ls 0.004312310905854931 0.007497354260061673 auc 0.9990854903100037 0.9898431638454348
threat 0.05 0.6 4
this class avg train 0.004021635718901568 avg val 0.006918724346347471
this class auc train 0.9992895168887236 auc val 0.9948578115535328
0 fold: ls 0.04669101221216726 0.049885251945004136 auc 0.9929883934369703 0.9914292836850717
1 fold: ls 0.04846292297732719 0.053628728602552134 auc 0.992340509136466 0.9897956409424158
2 fold: ls 0.04931522080326125 0.05492467615533997 auc 0.9920770129704343 0.9889534514085709
3 fold: ls 0.045901180019375105 0.051494417280043565 auc 0.9932617437219922 0.9905819494125796
4 fold: ls 0.048241973312669544 0.055015570647356224 auc 0.9924225397087889 0.9891893967559405
5 fold: ls 0.047063627595908396 0.05195763204868201 auc 0.9928504638580463 0.9907650454624095
6 fold: ls 0.047102060797921355 0.05646861011914467 auc 0.9928073000952728 0.989059138444